In [1]:
# This model is created for easy translation from english to hindi

In [2]:
# Imports
import numpy as np
import pandas as pd
import re
import string
from string import digits

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# tensorflow and keras
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

2023-05-25 18:35:48.572745: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 18:35:48.632930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 18:35:49.723814: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Data Preparation

In [3]:
df = pd.read_csv('./Hindi_English_Truncated_Corpus.csv')

In [4]:
df.drop(columns='source')

,english,hindi
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...
35494,The older sons joined their father 's business...,उनके बड़े पुत्र फ्रांस में अपने पिता की व्यावस...
35495,Best collection of some special hindi poetry.,हिन्दी काव्य के उत्कृष्ट रचनाओं का उत्तम संग्रह
35496,And it took about a year.,इसमें तकरीबन एक साल लगा ।
35497,Put yourself in the shoes,अब आप एक अमेरिकेन सेनिक की जगह से सोची ये


In [5]:
df.isnull().sum()

source     0
english    0
hindi      0
dtype: int64

In [6]:
df['hindi'] = df['hindi'].astype(str)
df['english'] = df['english'].astype(str)

In [7]:
df['hindi'] = df['hindi'].str.strip()  # Hindi has no capital letters
df['english'] = df['english'].str.strip().str.lower()

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df=df[~df['english'].isnull()]

In [10]:
# Remove quotes
df['english']=df['english'].apply(lambda x: re.sub("'", '', x))
df['hindi']=df['hindi'].apply(lambda x: re.sub("'", '', x))

In [11]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
df['english']=df['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df['hindi']=df['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [12]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
df['english']=df['english'].apply(lambda x: x.translate(remove_digits))
df['hindi']=df['hindi'].apply(lambda x: x.translate(remove_digits))

df['hindi']=df['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
df['english']=df['english'].apply(lambda x: x.strip())
df['hindi']=df['hindi'].apply(lambda x: x.strip())
df['english']=df['english'].apply(lambda x: re.sub(" +", " ", x))
df['hindi']=df['hindi'].apply(lambda x: re.sub(" +", " ", x))

In [13]:
# Add start and end tokens to target sequences
df['hindi'] = df['hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in df['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in df['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [15]:
df

,source,english,hindi
0,ted,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...
1,ted,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...
2,indic2012,this percentage is even greater than the perce...,START_ यह प्रतिशत भारत में हिन्दुओं प्रतिशत से...
3,ted,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...
4,indic2012,the ending portion of these vedas is called up...,START_ इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता...
...,...,...,...
35494,tides,the older sons joined their father s business ...,START_ उनके बड़े पुत्र फ्रांस में अपने पिता की...
35495,indic2012,best collection of some special hindi poetry,START_ हिन्दी काव्य के उत्कृष्ट रचनाओं का उत्त...
35496,ted,and it took about a year,START_ इसमें तकरीबन एक साल लगा । _END
35497,ted,put yourself in the shoes,START_ अब आप एक अमेरिकेन सेनिक की जगह से सोची ...


In [16]:
len(all_eng_words), len(all_hindi_words)

(36798, 44493)

In [17]:
df['length_eng_sentence']=df['english'].apply(lambda x:len(x.split(" ")))
df['length_hin_sentence']=df['hindi'].apply(lambda x:len(x.split(" ")))

In [18]:
df=df[df['length_eng_sentence']<=20]
df=df[df['length_hin_sentence']<=20]

In [19]:
df.shape

(23095, 5)

In [20]:
print("maximum length of Hindi Sentence ",max(df['length_hin_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [21]:
max_length_src=max(df['length_hin_sentence'])
max_length_tar=max(df['length_eng_sentence'])

In [22]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(36798, 44493)

In [37]:
num_encoder_tokens += 1 #for zero padding
num_decoder_tokens += 1 #for zero padding

In [38]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [39]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [40]:
df = shuffle(df)
df.head(10)

,source,english,hindi,length_eng_sentence,length_hin_sentence
33449,indic2012,upto year pakistans economy had been increasin...,START_ सन् तक पाकिस्तान की अर्थव्यवस्था प्रतिश...,8,16
21778,indic2012,in uttar pradesh maximum lok sabha members are...,START_ उत्तर प्रदेश से सर्वाधिक लोक सभा व राज्...,13,16
16223,indic2012,ramrajya become idol,START_ रामराज्य एक आदर्श बन गया। _END,3,7
29317,ted,its that reeeeeeeeach out the physical contortion,START_ यह पसारना हैं शारीरिक विकृति _END,7,7
33392,indic2012,on this day for worship of yama deepak is infl...,START_ इस दिन यम पूजा हेतु दीपक जलाए जाते हैं।...,10,11
31615,ted,that makes us beautiful,START_ जो हमें सुन्दर बनाती हैं _END,4,7
21247,tides,it takes seconds for the outgoing blood to ret...,START_ हृदय से बाहर निकले रक़्त को वापस आने मे...,12,16
33773,ted,he said “do you know theres a solarelectrified...,START_ उसने कहा “आपको पता है कि सियरा ल्योन मे...,12,16
338,ted,if i spin this pulley the small ones going to ...,START_ अगर मैं इस चक्की को घुमाऊं तो छोटा वाला...,13,14
32905,ted,can stand out in the way that these things have,START_ यहाँ मशहूर हो सकते हैं। _END,10,7


In [41]:
X_train, X_test, y_train, y_test = train_test_split(df['english'], df['hindi'], test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((18476,), (4619,))

In [42]:
# Let's save this data
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [43]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

## Encoder Decoder Architecture

In [44]:
embedding_dim = 100

In [45]:
# Encoder
encoder_inputs = Input(shape=(None,)) # Defines an input layer for the encoder. The Input function is from the Keras API, and shape=(None,) specifies that the input sequence length can vary (indicated by None) and the dimensionality of each input element is unspecified (to be determined later).
enc_emb =  Embedding(num_encoder_tokens, embedding_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(embedding_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [46]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, embedding_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we can use them in inference.
decoder_lstm = LSTM(embedding_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [47]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [48]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 100)    3679900     ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 100)    4449500     ['input_4[0][0]']                
                                                                                            

In [49]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [50]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/tmp/ipykernel_16712/2108290721.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/50


2023-05-25 18:38:19.162486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


144/144 [==============================] - ETA: 0s - loss: 8.3075

2023-05-25 18:45:07.096848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


144/144 [==============================] - 467s 3s/step - loss: 8.3075 - val_loss: 7.2728
Epoch 2/50
144/144 [==============================] - 458s 3s/step - loss: 7.1542 - val_loss: 7.1211
Epoch 3/50
144/144 [==============================] - 456s 3s/step - loss: 7.0347 - val_loss: 7.0505
Epoch 4/50
144/144 [==============================] - 457s 3s/step - loss: 6.9701 - val_loss: 7.0162
Epoch 5/50
144/144 [==============================] - 457s 3s/step - loss: 6.9339 - val_loss: 6.9934
Epoch 6/50
144/144 [==============================] - 457s 3s/step - loss: 6.9081 - val_loss: 6.9788
Epoch 7/50
144/144 [==============================] - 473s 3s/step - loss: 6.8853 - val_loss: 6.9640
Epoch 8/50
144/144 [==============================] - 456s 3s/step - loss: 6.8674 - val_loss: 6.9528
Epoch 9/50
144/144 [==============================] - 464s 3s/step - loss: 6.8521 - val_loss: 6.9404
Epoch 10/50
144/144 [==============================] - 480s 3s/step - loss: 6.8365 - val_loss: 6.9325


In [51]:
model.save_weights('nmt_weights.h5')

In [52]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(embedding_dim,))
decoder_state_input_c = Input(shape=(embedding_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [53]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [54]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [55]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 26ms/step
Input English sentence: we see it in supermarkets on book covers
Actual Hindi Translation:  हमने यह पुस्तक कवर पर सुपरमार्केट में देख़ा हैं। 
Predicted Hindi Translation:  हम एक एक एक एक एक एक एक एक भी भी भी 


In [56]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 33ms/step
Input English sentence: and gain a better understanding of our landscape
Actual Hindi Translation:  और हमारे भुदेश्य की एक बेहतर समझ हासिल कर सकते हैं 
Predicted Hindi Translation:  और और हम एक एक एक एक एक एक एक भी भी भी 


In [ ]:
# predictoins are somewhat inaccurate maybe because of decreased size of dataset